# F1 Aerodynamics: Complex Flow Phenomena with Spectral Analysis

This notebook explores various flow regimes to **identify and visualize complex phenomena**:
- **Velocity & Pressure fields** - Primary flow structures
- **Vorticity fields** - Vortex formation and shedding
- **Energy spectra** - Turbulent cascade (Kolmogorov -5/3 law)
- **2D spectral maps** - Wavenumber space analysis
- **Flow regime transitions** - Laminar → Turbulent

In [5]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import numpy as np
from lbm_core import LBMSolver
from boundaries import TunnelBoundaries
from analysis import plot_complexity_dashboard
from aerodynamics import calculate_lift_drag, check_ground_effect
import json
from datetime import datetime
from pathlib import Path
from scipy import signal
from scipy.optimize import curve_fit

## Configuration: Parameter Space for Phenomena Discovery

In [6]:
# --- BASE CONFIGURATION ---
NX, NY = 500, 200
GROUND_TYPE = "no_slip"
SIMULATION_STEPS = 5000  # Longer runs for spectral convergence
CHECKPOINT_INTERVAL = 100
SPECTRUM_INTERVAL = 500  # Compute spectra less frequently (expensive)

# --- PARAMETER SWEEP FOR PHENOMENA ---
RIDE_HEIGHTS = [35]  # Ground effect strength variation
REYNOLDS_NUMBERS = [20000]  # Laminar → Turbulent

# --- OUTPUT CONFIGURATION ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_dir = Path(f"flow_phenomena_{timestamp}")
results_dir.mkdir(exist_ok=True)

(results_dir / "snapshots").mkdir(exist_ok=True)
(results_dir / "animations").mkdir(exist_ok=True)
(results_dir / "vorticity").mkdir(exist_ok=True)
(results_dir / "pressure").mkdir(exist_ok=True)
(results_dir / "spectra").mkdir(exist_ok=True)
(results_dir / "timeseries").mkdir(exist_ok=True)
(results_dir / "analysis").mkdir(exist_ok=True)

print(f"Phenomena exploration results → {results_dir}")
print(f"Total configurations: {len(RIDE_HEIGHTS) * len(REYNOLDS_NUMBERS)}")

Phenomena exploration results → flow_phenomena_20260126_202808
Total configurations: 1


## Advanced Phenomena Tracking System

In [7]:
class FlowPhenomenaTracker:
    """Track and analyze complex flow phenomena with spectral analysis"""
    
    def __init__(self, ride_height, reynolds, nx, ny):
        self.ride_height = ride_height
        self.reynolds = reynolds
        self.nx = nx
        self.ny = ny
        self.timestamp = datetime.now().isoformat()
        
        # Evolution tracking
        self.velocity_snapshots = []
        self.vorticity_snapshots = []
        self.pressure_snapshots = []
        self.snapshot_steps = []
        
        # Spectral data
        self.energy_spectra_1d = []      # 1D radial energy spectra
        self.energy_spectra_2d = []      # Full 2D spectral maps
        self.spectrum_steps = []
        
        # Time series
        self.lift_history = []
        self.drag_history = []
        self.step_history = []
        self.max_velocity_history = []
        self.vorticity_magnitude = []
        
        # Phenomena indicators
        self.vortex_shedding_detected = False
        self.shedding_frequency = None
        self.turbulence_intensity = None
        self.kolmogorov_slope = None
        self.inertial_range_quality = None
    
    def calculate_pressure_field(self, solver, mask):
        """Calculate pressure field from distribution functions"""
        # Pressure is proportional to density in LBM
        rho = np.sum(solver.f, axis=2)
        
        # Pressure coefficient: C_p = (p - p_inf) / (0.5 * rho_inf * u_inf^2)
        rho_inf = 1.0  # Reference density
        u_inf = solver.u_inlet
        
        pressure = (rho - rho_inf) / (0.5 * rho_inf * u_inf**2)
        pressure[mask] = np.nan
        
        return pressure
    
    def calculate_vorticity(self, velocity_field, mask):
        """Calculate vorticity field (ω = ∂v/∂x - ∂u/∂y)"""
        u = velocity_field[:,:,0]
        v = velocity_field[:,:,1]
        
        du_dy = np.gradient(u, axis=0)
        dv_dx = np.gradient(v, axis=1)
        
        vorticity = dv_dx - du_dy
        vorticity[mask] = np.nan
        
        return vorticity
    
    def compute_energy_spectrum(self, velocity_field, mask):
        """Compute turbulent kinetic energy spectrum E(k)"""
        # Extract velocity components
        u = velocity_field[:,:,0].copy()
        v = velocity_field[:,:,1].copy()
        
        # Mask out solid regions
        u[mask] = 0
        v[mask] = 0
        
        # Compute 2D FFT of velocity components
        u_fft = np.fft.fft2(u)
        v_fft = np.fft.fft2(v)
        
        # Kinetic energy in Fourier space
        energy_2d = 0.5 * (np.abs(u_fft)**2 + np.abs(v_fft)**2)
        
        # Get wavenumber grids
        kx = np.fft.fftfreq(self.nx) * self.nx
        ky = np.fft.fftfreq(self.ny) * self.ny
        kx_grid, ky_grid = np.meshgrid(kx, ky)
        k_mag = np.sqrt(kx_grid**2 + ky_grid**2)
        
        # Shift zero frequency to center for 2D visualization
        energy_2d_shifted = np.fft.fftshift(energy_2d)
        
        # Radially average to get 1D spectrum E(k)
        k_bins = np.arange(0.5, min(self.nx, self.ny)//2, 1.0)
        k_centers = 0.5 * (k_bins[:-1] + k_bins[1:])
        energy_1d = np.zeros(len(k_centers))
        
        for i, k_center in enumerate(k_centers):
            mask_ring = (k_mag >= k_bins[i]) & (k_mag < k_bins[i+1])
            if np.any(mask_ring):
                energy_1d[i] = np.mean(energy_2d[mask_ring])
        
        return k_centers, energy_1d, energy_2d_shifted
    
    def fit_kolmogorov_slope(self, k, E_k):
        """Fit power law to inertial range and extract slope"""
        # Take only positive energies
        valid = (E_k > 0) & (k > 1)
        k_valid = k[valid]
        E_valid = E_k[valid]
        
        if len(k_valid) < 5:
            return None, None
        
        # Log-log fit: log(E) = slope * log(k) + intercept
        log_k = np.log10(k_valid)
        log_E = np.log10(E_valid)
        
        # Fit in the inertial range (middle decades)
        mid_start = len(log_k) // 4
        mid_end = 3 * len(log_k) // 4
        
        if mid_end - mid_start < 3:
            mid_start = 0
            mid_end = len(log_k)
        
        coeffs = np.polyfit(log_k[mid_start:mid_end], log_E[mid_start:mid_end], 1)
        slope = coeffs[0]
        
        # Quality metric: how well does it fit -5/3?
        quality = abs(slope - (-5/3))
        
        return slope, quality
    
    def add_snapshot(self, step, solver, bounds_mask):
        """Capture full flow state for later analysis"""
        self.snapshot_steps.append(step)
        
        # Velocity magnitude
        v_mag = np.sqrt(solver.u[:,:,0]**2 + solver.u[:,:,1]**2)
        v_mag[bounds_mask] = np.nan
        self.velocity_snapshots.append(v_mag.copy())
        
        # Vorticity
        vorticity = self.calculate_vorticity(solver.u, bounds_mask)
        self.vorticity_snapshots.append(vorticity)
        self.vorticity_magnitude.append(np.nansum(np.abs(vorticity)))
        
        # Pressure
        pressure = self.calculate_pressure_field(solver, bounds_mask)
        self.pressure_snapshots.append(pressure)
    
    def add_spectrum(self, step, solver, bounds_mask):
        """Compute and store energy spectrum"""
        k, E_k, E_2d = self.compute_energy_spectrum(solver.u, bounds_mask)
        
        self.spectrum_steps.append(step)
        self.energy_spectra_1d.append((k, E_k))
        self.energy_spectra_2d.append(E_2d)
    
    def add_step_data(self, step, lift, drag, max_vel):
        """Record time series data"""
        self.step_history.append(step)
        self.lift_history.append(lift)
        self.drag_history.append(drag)
        self.max_velocity_history.append(max_vel)
    
    def analyze_phenomena(self):
        """Detect and characterize flow phenomena"""
        # Vortex shedding detection via FFT of forces
        if len(self.lift_history) > 500:
            lift_signal = np.array(self.lift_history[-1000:])
            lift_signal = lift_signal - np.mean(lift_signal)
            
            fft = np.fft.fft(lift_signal)
            freqs = np.fft.fftfreq(len(lift_signal))
            power = np.abs(fft)**2
            
            positive_freqs = freqs[1:len(freqs)//2]
            positive_power = power[1:len(power)//2]
            
            if len(positive_power) > 0:
                peak_idx = np.argmax(positive_power)
                peak_freq = positive_freqs[peak_idx]
                peak_power = positive_power[peak_idx]
                
                if peak_power > 5 * np.mean(positive_power):
                    self.vortex_shedding_detected = True
                    self.shedding_frequency = abs(peak_freq)
        
        # Turbulence intensity
        if len(self.max_velocity_history) > 100:
            vel_array = np.array(self.max_velocity_history[-500:])
            mean_vel = np.mean(vel_array)
            if mean_vel > 0:
                self.turbulence_intensity = np.std(vel_array) / mean_vel
        
        # Analyze energy cascade (using last spectrum)
        if len(self.energy_spectra_1d) > 0:
            k, E_k = self.energy_spectra_1d[-1]
            slope, quality = self.fit_kolmogorov_slope(k, E_k)
            self.kolmogorov_slope = slope
            self.inertial_range_quality = quality
    
    def get_phenomena_summary(self):
        """Return dictionary of detected phenomena"""
        return {
            'ride_height': self.ride_height,
            'reynolds': self.reynolds,
            'vortex_shedding': self.vortex_shedding_detected,
            'shedding_frequency': float(self.shedding_frequency) if self.shedding_frequency else None,
            'turbulence_intensity': float(self.turbulence_intensity) if self.turbulence_intensity else None,
            'kolmogorov_slope': float(self.kolmogorov_slope) if self.kolmogorov_slope else None,
            'inertial_range_quality': float(self.inertial_range_quality) if self.inertial_range_quality else None,
            'mean_vorticity': float(np.mean(self.vorticity_magnitude)) if self.vorticity_magnitude else None,
        }
    
    def save(self, base_dir):
        """Save all phenomena data"""
        base_dir = Path(base_dir)
        prefix = f"rh{self.ride_height}_re{self.reynolds}"
        
        # Phenomena summary
        with open(base_dir / "analysis" / f"{prefix}_phenomena.json", 'w') as f:
            json.dump(self.get_phenomena_summary(), f, indent=2)
        
        # Time series
        np.savez_compressed(
            base_dir / "timeseries" / f"{prefix}_evolution.npz",
            steps=self.step_history,
            lift=self.lift_history,
            drag=self.drag_history,
            max_velocity=self.max_velocity_history,
            vorticity_magnitude=self.vorticity_magnitude
        )
        
        # Snapshots
        np.savez_compressed(
            base_dir / "snapshots" / f"{prefix}_snapshots.npz",
            velocity_fields=np.array(self.velocity_snapshots),
            vorticity_fields=np.array(self.vorticity_snapshots),
            pressure_fields=np.array(self.pressure_snapshots),
            snapshot_steps=self.snapshot_steps
        )
        
        # Spectral data
        if len(self.energy_spectra_1d) > 0:
            spectra_dict = {
                'steps': self.spectrum_steps,
                'k_values': [k for k, _ in self.energy_spectra_1d],
                'E_k_values': [E for _, E in self.energy_spectra_1d],
                'E_2d_maps': np.array(self.energy_spectra_2d)
            }
            np.savez_compressed(
                base_dir / "spectra" / f"{prefix}_energy_spectra.npz",
                **spectra_dict
            )
        
        print(f"  💾 Saved: {prefix}")


def create_geometry(bounds, ride_height):
    """Create F1 car geometry"""
    bounds.add_ground(type=GROUND_TYPE)
    
    bounds.add_f1_wing_proxy(
        x_pos=150,
        height=ride_height,
        length=60,
        slope=0.45
    )
    
    bounds.add_rectangular_obstacle(
        x_start=210,
        y_start=ride_height + 1,
        length=120,
        height=20
    )
    
    bounds.add_reverse_triangle(
        x_pos=240,
        height=ride_height + 20,
        length=90,
        slope=2/9
    )
    
    bounds.add_f1_wing_proxy(
        x_pos=300,
        height=ride_height + 30,
        length=30,
        slope=0.2
    )

## Run Phenomena Exploration with Spectral Analysis

In [8]:
all_trackers = []

total_sims = len(RIDE_HEIGHTS) * len(REYNOLDS_NUMBERS)
sim_counter = 0

print("="*70)
print("EXPLORING FLOW PHENOMENA WITH SPECTRAL ANALYSIS")
print("="*70)

for reynolds in REYNOLDS_NUMBERS:
    for ride_height in RIDE_HEIGHTS:
        sim_counter += 1
        print(f"\n[{sim_counter}/{total_sims}] Re={reynolds}, Ride Height={ride_height}")
        print("-" * 70)
        
        tracker = FlowPhenomenaTracker(ride_height, reynolds, NX, NY)
        
        solver = LBMSolver(NX, NY, reynolds, u_inlet=0.1)
        bounds = TunnelBoundaries(NX, NY)
        create_geometry(bounds, ride_height)
        
        relevant_x_start = 145
        relevant_x_end = 335
        relevant_y_start = 5
        relevant_y_end = min(75, ride_height + 60)
        
        print(f"  Running {SIMULATION_STEPS} steps...")
        
        for step in range(SIMULATION_STEPS):
            solver.collide_and_stream(bounds.mask)
            bounds.apply_inlet_outlet(solver)
            
            # Track forces frequently
            if step % 10 == 0:
                fx, fy = calculate_lift_drag(
                    solver, bounds,
                    x_start=relevant_x_start,
                    x_end=relevant_x_end,
                    y_start=relevant_y_start,
                    y_end=relevant_y_end
                )
                max_vel = np.max(np.sqrt(solver.u[:,:,0]**2 + solver.u[:,:,1]**2))
                tracker.add_step_data(step, fy, fx, max_vel)
            
            # Capture flow snapshots
            if step % CHECKPOINT_INTERVAL == 0 or step == SIMULATION_STEPS - 1:
                tracker.add_snapshot(step, solver, bounds.mask)
                
                if step % (CHECKPOINT_INTERVAL * 5) == 0:
                    print(f"    Step {step}: Snapshot saved")
            
            # Compute energy spectrum (expensive, do less frequently)
            if step % SPECTRUM_INTERVAL == 0 and step > 0:
                tracker.add_spectrum(step, solver, bounds.mask)
                print(f"    Step {step}: Energy spectrum computed")
        
        # Final spectrum
        tracker.add_spectrum(SIMULATION_STEPS, solver, bounds.mask)
        
        # Analyze phenomena
        tracker.analyze_phenomena()
        summary = tracker.get_phenomena_summary()
        
        print(f"\n  🔍 PHENOMENA DETECTED:")
        print(f"     Vortex Shedding: {'YES' if summary['vortex_shedding'] else 'NO'}")
        if summary['shedding_frequency']:
            print(f"     Shedding Frequency: {summary['shedding_frequency']:.6f}")
        if summary['turbulence_intensity']:
            print(f"     Turbulence Intensity: {summary['turbulence_intensity']:.4f}")
        if summary['kolmogorov_slope']:
            print(f"     Energy Cascade Slope: {summary['kolmogorov_slope']:.3f} (Kolmogorov: -1.67)")
            print(f"     Cascade Quality: {summary['inertial_range_quality']:.3f} (lower is better)")
        
        tracker.save(results_dir)
        all_trackers.append(tracker)

print("\n" + "="*70)
print("PHENOMENA EXPLORATION COMPLETE")
print("="*70)

EXPLORING FLOW PHENOMENA WITH SPECTRAL ANALYSIS

[1/1] Re=20000, Ride Height=35
----------------------------------------------------------------------
  Running 5000 steps...
    Step 0: Snapshot saved
    Step 500: Snapshot saved
    Step 500: Energy spectrum computed
    Step 1000: Snapshot saved
    Step 1000: Energy spectrum computed
    Step 1500: Snapshot saved
    Step 1500: Energy spectrum computed
    Step 2000: Snapshot saved
    Step 2000: Energy spectrum computed
    Step 2500: Snapshot saved
    Step 2500: Energy spectrum computed
    Step 3000: Snapshot saved
    Step 3000: Energy spectrum computed
    Step 3500: Snapshot saved
    Step 3500: Energy spectrum computed
    Step 4000: Snapshot saved
    Step 4000: Energy spectrum computed
    Step 4500: Snapshot saved
    Step 4500: Energy spectrum computed

  🔍 PHENOMENA DETECTED:
     Vortex Shedding: NO
     Turbulence Intensity: 0.1008
     Energy Cascade Slope: -3.918 (Kolmogorov: -1.67)
     Cascade Quality: 2.252 (low

## Visualize Velocity & Pressure Fields (Like Your Examples)

In [9]:
print("Creating velocity + pressure field visualizations...\n")

for tracker in all_trackers:
    if len(tracker.velocity_snapshots) == 0:
        continue
    
    # Use final snapshot
    velocity = tracker.velocity_snapshots[-1]
    pressure = tracker.pressure_snapshots[-1]
    step = tracker.snapshot_steps[-1]
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Velocity field (magma colormap like your example)
    ax = axes[0]
    im = ax.imshow(velocity, origin='lower', cmap='magma', vmin=0, vmax=0.15, interpolation='bilinear')
    ax.set_title(f'Velocity Field | Re={tracker.reynolds} | Step {step}', 
                 fontsize=14, fontweight='bold')
    ax.set_xlabel('x (lattice units)', fontsize=11)
    ax.set_ylabel('y (lattice units)', fontsize=11)
    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02)
    cbar.set_label('Velocity Magnitude |u|', fontsize=11)
    
    # Pressure field (RdBu colormap like your example)
    ax = axes[1]
    vmax_p = np.nanpercentile(np.abs(pressure), 99)
    im = ax.imshow(pressure, origin='lower', cmap='RdBu_r', vmin=-vmax_p, vmax=vmax_p, interpolation='bilinear')
    ax.set_title(f'Pressure Field | Re={tracker.reynolds} | Step {step}', 
                 fontsize=14, fontweight='bold')
    ax.set_xlabel('x (lattice units)', fontsize=11)
    ax.set_ylabel('y (lattice units)', fontsize=11)
    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02)
    cbar.set_label('Pressure Coefficient $C_p$', fontsize=11)
    
    plt.tight_layout()
    filename = f"rh{tracker.ride_height}_re{tracker.reynolds}_vel_pressure.png"
    plt.savefig(results_dir / "pressure" / filename, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"  📊 {filename}")

print("\nVelocity + Pressure visualizations complete!")

Creating velocity + pressure field visualizations...

  📊 rh35_re20000_vel_pressure.png

Velocity + Pressure visualizations complete!


## Energy Spectra: 1D Cascade & 2D Maps (Like Your Examples)

In [10]:
print("Creating energy spectrum visualizations...\n")

for tracker in all_trackers:
    if len(tracker.energy_spectra_1d) == 0:
        print(f"  ⚠ No spectra for Re={tracker.reynolds}, RH={tracker.ride_height}")
        continue
    
    # Get final spectrum
    k, E_k = tracker.energy_spectra_1d[-1]
    E_2d = tracker.energy_spectra_2d[-1]
    step = tracker.spectrum_steps[-1]
    
    fig = plt.figure(figsize=(16, 6))
    
    # Left: 1D Energy Spectrum with theoretical slopes
    ax1 = plt.subplot(1, 2, 1)
    
    # Plot simulation data
    valid = (E_k > 0) & (k > 0.5)
    ax1.loglog(k[valid], E_k[valid], 'b-', linewidth=2.5, label='Simulation')
    
    # Theoretical Kolmogorov k^(-5/3)
    k_theory = k[valid]
    C_kolm = E_k[valid][len(k_theory)//3] * (k_theory[len(k_theory)//3] ** (5/3))
    E_kolm = C_kolm * k_theory ** (-5/3)
    ax1.loglog(k_theory, E_kolm, 'g--', linewidth=2, alpha=0.7, label='Kolmogorov $k^{-5/3}$')
    
    # Theoretical Kraichnan k^(-3) (for 2D turbulence)
    C_kraich = E_k[valid][len(k_theory)//3] * (k_theory[len(k_theory)//3] ** 3)
    E_kraich = C_kraich * k_theory ** (-3)
    ax1.loglog(k_theory, E_kraich, 'r:', linewidth=2, alpha=0.7, label='Kraichnan $k^{-3}$')
    
    ax1.set_xlabel('Wave Number $k$', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Energy $E(k)$', fontsize=13, fontweight='bold')
    ax1.set_title('Energy Spectrum', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11, loc='upper right')
    ax1.grid(True, alpha=0.3, which='both', linestyle=':')
    
    # Add fitted slope info
    if tracker.kolmogorov_slope:
        textstr = f'Slope: {tracker.kolmogorov_slope:.2f}\n'
        if tracker.kolmogorov_slope < -1.5:
            textstr += 'Close to Kolmogorov $k^{{-5/3}}$ - inverse cascade'
        else:
            textstr += 'Deviation from classical cascade'
        ax1.text(0.05, 0.05, textstr, transform=ax1.transAxes, 
                fontsize=10, verticalalignment='bottom',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Right: 2D Energy Spectrum (wavenumber space)
    ax2 = plt.subplot(1, 2, 2)
    
    # Log scale for better visualization
    E_2d_log = np.log10(E_2d + 1e-10)  # Add small value to avoid log(0)
    
    im = ax2.imshow(E_2d_log, origin='lower', cmap='hot', 
                    extent=[0, tracker.nx, 0, tracker.ny], aspect='auto', interpolation='bilinear')
    ax2.set_xlabel('$k_x$', fontsize=13, fontweight='bold')
    ax2.set_ylabel('$k_y$', fontsize=13, fontweight='bold')
    ax2.set_title('2D Energy Spectrum', fontsize=14, fontweight='bold')
    
    cbar = plt.colorbar(im, ax=ax2, pad=0.02)
    cbar.set_label('$\log_{10}$(Energy)', fontsize=11)
    
    # Overall title
    fig.suptitle(f'Turbulent Energy Analysis - Re={tracker.reynolds}, Ride Height={tracker.ride_height}, Step={step}',
                 fontsize=15, fontweight='bold', y=0.98)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    filename = f"rh{tracker.ride_height}_re{tracker.reynolds}_energy_spectrum.png"
    plt.savefig(results_dir / "spectra" / filename, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"  📈 {filename}")

print("\nEnergy spectrum visualizations complete!")

Creating energy spectrum visualizations...



<>:63: SyntaxWarning: invalid escape sequence '\l'
<>:63: SyntaxWarning: invalid escape sequence '\l'
C:\Users\Nicholas Miot\AppData\Local\Temp\ipykernel_3292\4068897910.py:63: SyntaxWarning: invalid escape sequence '\l'
  cbar.set_label('$\log_{10}$(Energy)', fontsize=11)


  📈 rh35_re20000_energy_spectrum.png

Energy spectrum visualizations complete!


## Vorticity Evolution

In [11]:
print("Creating vorticity visualizations...\n")

for tracker in all_trackers:
    n_snapshots = len(tracker.vorticity_snapshots)
    if n_snapshots == 0:
        continue
    
    indices = [0, n_snapshots//4, n_snapshots//2, -1]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    axes = axes.flatten()
    
    all_vort = np.concatenate([v.flatten() for v in tracker.vorticity_snapshots])
    vmax = np.nanpercentile(np.abs(all_vort), 99)
    
    for i, idx in enumerate(indices):
        ax = axes[i]
        vorticity = tracker.vorticity_snapshots[idx]
        step = tracker.snapshot_steps[idx]
        
        im = ax.imshow(vorticity, origin='lower', cmap='RdBu_r', 
                      vmin=-vmax, vmax=vmax, interpolation='bilinear')
        ax.set_title(f'Step {step} - Vorticity Field', fontsize=12, fontweight='bold')
        ax.set_xlabel('x (lattice units)')
        ax.set_ylabel('y (lattice units)')
        plt.colorbar(im, ax=ax, label='Vorticity $\omega$')
    
    fig.suptitle(f'Vortex Evolution - Re={tracker.reynolds}, Ride Height={tracker.ride_height}',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    filename = f"rh{tracker.ride_height}_re{tracker.reynolds}_vorticity_evolution.png"
    plt.savefig(results_dir / "vorticity" / filename, dpi=200, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"  🌀 {filename}")

print("\nVorticity visualizations complete!")

Creating vorticity visualizations...



<>:26: SyntaxWarning: invalid escape sequence '\o'
<>:26: SyntaxWarning: invalid escape sequence '\o'
C:\Users\Nicholas Miot\AppData\Local\Temp\ipykernel_3292\2078824794.py:26: SyntaxWarning: invalid escape sequence '\o'
  plt.colorbar(im, ax=ax, label='Vorticity $\omega$')


  🌀 rh35_re20000_vorticity_evolution.png

Vorticity visualizations complete!


## Summary: Identified Phenomena

In [12]:
phenomena_list = [t.get_phenomena_summary() for t in all_trackers]

with open(results_dir / "phenomena_summary.json", 'w') as f:
    json.dump(phenomena_list, f, indent=2)

print("="*70)
print("PHENOMENA DISCOVERY SUMMARY")
print("="*70)
print(f"\nTotal configurations: {len(all_trackers)}")

shedding_cases = [p for p in phenomena_list if p['vortex_shedding']]
print(f"\n🌀 Vortex shedding in {len(shedding_cases)}/{len(phenomena_list)} cases")

cascade_cases = [p for p in phenomena_list if p['kolmogorov_slope'] is not None]
if cascade_cases:
    print(f"\n📉 Energy Cascade Analysis ({len(cascade_cases)} cases):")
    for p in cascade_cases:
        quality_str = "Good" if p['inertial_range_quality'] < 0.5 else "Moderate" if p['inertial_range_quality'] < 1.0 else "Poor"
        print(f"  • Re={p['reynolds']:5d}, RH={p['ride_height']:2d}: slope={p['kolmogorov_slope']:+.3f} ({quality_str})")

print(f"\n📁 Results saved to: {results_dir}")
print(f"\nGenerated outputs:")
print(f"  • {len(list((results_dir / 'pressure').glob('*.png')))} velocity+pressure visualizations")
print(f"  • {len(list((results_dir / 'spectra').glob('*.png')))} energy spectra plots")
print(f"  • {len(list((results_dir / 'vorticity').glob('*.png')))} vorticity visualizations")
print(f"  • {len(list((results_dir / 'snapshots').glob('*.npz')))} flow evolution datasets")

PHENOMENA DISCOVERY SUMMARY

Total configurations: 1

🌀 Vortex shedding in 0/1 cases

📉 Energy Cascade Analysis (1 cases):
  • Re=20000, RH=35: slope=-3.918 (Poor)

📁 Results saved to: flow_phenomena_20260126_202808

Generated outputs:
  • 1 velocity+pressure visualizations
  • 1 energy spectra plots
  • 1 vorticity visualizations
  • 1 flow evolution datasets
